This notebook performs a comprehensive becnhmark suit for the inference procedure for the double pendulum system using the RxInfer framework.

_Author: Dmitry Bagaev_

In [1]:
using DrWatson

In [2]:
@quickactivate "RxInferThesisExperiments"

In [3]:
using RxInferThesisExperiments, ForneyLab, StaticArrays, Plots, PGFPlotsX, LaTeXStrings
using LinearAlgebra, StableRNGs, Random, BenchmarkTools, ColorSchemes, Dates, DataFrames

In [4]:
const bfolder = datadir("nlds", "forneylab")

"/Users/bvdmitri/.julia/dev/thesis/data/nlds/forneylab"

In [5]:
# Pregenerate paths for benchmark data
mkpath(bfolder);

In [6]:
# Create default environment with default parameters
const environment = DoublePendulum()

DoublePendulum()

In [7]:
# Define state-transition function, uses RK4 method internally, see the `src/` folder
f(state) = state_transition(environment)(state)

f (generic function with 1 method)

In [8]:
# Include the model specification
include(srcdir("models", "forneylab", "doublependulum.jl"));

In [9]:
function run_benchmark(params)
    @unpack T, niterations, seed = params
    
    states, observations = rand(StableRNG(seed), environment, T);
    model    = double_pendulum(T; force = false);
    result   = run_inference(model, observations; iterations = niterations)
    e_states = extract_posteriors(T, result)
    amse     = compute_amse(states, e_states)
    
    benchmark_modelcreation = @benchmark double_pendulum($T; force = true);
    
    benchmark_inference = @benchmark run_inference(model, observations; iterations = $niterations) setup=begin
        model = double_pendulum($T; force = false);
        states, observations = rand(StableRNG($seed), environment, $T);
    end
    
    emse = compute_emse(seed) do _seed
        local states, observations = rand(StableRNG(_seed), environment, T; random_start = true);
        local model    = double_pendulum(T; force = false);
        local result   = run_inference(model, observations; iterations = niterations)
        local e_states = extract_posteriors(T, result)
        return compute_amse(states, e_states)
    end
    
    output = @strdict T niterations seed states e_states observations amse emse benchmark_modelcreation benchmark_inference
    
    return output
end

run_benchmark (generic function with 1 method)

In [22]:
# Here we create a list of parameters we want to run our benchmarks with
benchmark_params = dict_list(Dict(
    "T"           => [ 10, 20, 30, 50, 100 ],
    "niterations" => [ 3 ],
    "seed"        => [ 42 ]
));

In [23]:
# First run maybe slow, you may track the progress in the terminal
# Subsequent runs will not create new benchmarks 
# but will reload it from data folder
benchmarks = map(benchmark_params) do params
    result, _ = produce_or_load(run_benchmark, bfolder, params; tag = false, force = false)
    return result
end;

[ Info: File /Users/bvdmitri/.julia/dev/thesis/data/nlds/forneylab/T=100_niterations=3_seed=42.jld2 does not exist. Producing it now...
[ Info: File /Users/bvdmitri/.julia/dev/thesis/data/nlds/forneylab/T=100_niterations=3_seed=42.jld2 saved.


In [26]:
sort(prepare_benchmarks_table(bfolder), [ :T ])

[ Info: Scanning folder /Users/bvdmitri/.julia/dev/thesis/data/nlds/forneylab for result files.
[ Info: Added 8 entries.


Row,T,seed,niterations,amse,emse,inference,creation
,Int64?,Int64?,Int64?,Float64?,Float64?,Tuple…?,Tuple…?
1,10,42,3,8.09207,7.10241,"(1.14833e6, 1.55756e6, 0.0)","(5.13339e9, 5.13339e9, 3.33827e7)"
2,10,42,5,8.0052,7.0327,"(1.95245e6, 2.94054e6, 0.0)","(5.09341e9, 5.09341e9, 3.50069e7)"
3,20,42,3,11.087,12.1779,"(2.70781e6, 3.52697e6, 0.0)","(1.09954e10, 1.09954e10, 6.62345e7)"
4,20,42,5,10.8625,12.0709,"(3.84851e6, 5.07834e6, 0.0)","(1.09624e10, 1.09624e10, 7.04102e7)"
5,30,42,3,9.23668,6.52354,"(3.62542e6, 4.71653e6, 0.0)","(1.59941e10, 1.59941e10, 9.59679e7)"
6,30,42,5,9.04062,6.45478,"(6.21228e6, 7.77057e6, 0.0)","(1.56399e10, 1.56399e10, 1.0033e8)"
7,50,42,3,7.23535,3.53501,"(5.97635e6, 8.76036e6, 0.0)","(2.75097e10, 2.75097e10, 1.65037e8)"
8,100,42,3,4.16957,1.28983,"(1.36045e7, 1.66293e7, 0.0)","(5.13783e10, 5.13783e10, 3.03572e8)"


# Versions

In [13]:
versioninfo()

Julia Version 1.9.0
Commit 8e630552924 (2023-05-07 11:25 UTC)
Platform Info:
  OS: macOS (x86_64-apple-darwin22.4.0)
  CPU: 12 × Intel(R) Core(TM) i7-8850H CPU @ 2.60GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-14.0.6 (ORCJIT, skylake)
  Threads: 2 on 12 virtual cores


In [14]:
] status

Project RxInferThesisExperiments v1.0.0
Status `~/.julia/dev/thesis/Project.toml`
  [6e4b80f9] BenchmarkTools v1.3.2
  [35d6a980] ColorSchemes v3.21.0
  [a93c6f00] DataFrames v1.5.0
⌃ [31c24e10] Distributions v0.25.94
  [634d3b9d] DrWatson v2.12.5
  [9fc3f58a] ForneyLab v0.12.0
  [f6369f11] ForwardDiff v0.10.35
  [7073ff75] IJulia v1.24.0
  [b964fa9f] LaTeXStrings v1.3.0
  [3bd65402] Optimisers v0.2.18
  [8314cec4] PGFPlotsX v1.6.0
  [e4faabce] PProf v2.2.2
⌃ [91a5bcdd] Plots v1.38.12
  [37e2e3b7] ReverseDiff v1.14.6
⌃ [86711068] RxInfer v2.10.4
  [860ef19b] StableRNGs v1.0.0
  [aedffcd0] Static v0.8.7
  [90137ffa] StaticArrays v1.5.25
  [fce5fe82] Turing v0.25.1
  [e88e6eb3] Zygote v0.6.61
  [37e2e46d] LinearAlgebra
  [9a3f8284] Random
Info Packages marked with ⌃ have new versions available and may be upgradable.
